In [1]:
from kfp.components import create_component_from_func

import sys
sys.path.insert(0, "..")
from constants import NAMESPACE, HOST
from utils.auth import get_session_cookie

In [2]:
EXPERIMENT_NAME = "tutorial"

In [3]:
def add(a: float, b: float) -> float:
    """Returns sum of two arguments"""
    return a + b

add_op = create_component_from_func(
    func=add,
    base_image='python:3.7', # Optional, with default is python:3.7
    output_component_file='../components/add/component.yaml', # Optional
    packages_to_install=['pandas==0.24'], # Optional
)

def minus(a: float, b: float) -> float:
    """Returns minus of two arguments"""
    print(a-b)
    return a - b

minus_op = create_component_from_func(
    func=minus,
    base_image='python:3.7', # Optional
    output_component_file='../components/minus/component.yaml', # Optional
    packages_to_install=['pandas==0.24'], # Optional
)

In [4]:
from kfp import dsl
import kfp

@dsl.pipeline(
    name='Condition',
    description='Run minus op if the condition is satisfied.'
)
def add_and_minus():
    """A sample pipeline showing condition."""

    add_task = add_op(1, 2)
    sum_output_ref = add_task.outputs['Output']
    
    with dsl.Condition(sum_output_ref>3):
        minus_task = minus_op(sum_output_ref, 4)

In [5]:
session_cookie = get_session_cookie()
client = kfp.Client(
    host=f"{HOST}/pipeline",
    cookies=f"authservice_session={session_cookie}",
    namespace=NAMESPACE,
)

{'authservice_session': 'MTY0NDc0NzEzNXxOd3dBTkRKSU1qZFJOVlZNV1ZOT1JVRkxURTVhVUZKUlFrTkZRMHhLU2pkV1JFZGFWa2hZTlVNelR6ZENVVUpIU1VWU1ZrSkZXRUU9fNg5pxigpUMGulthaeqMIMCG9KMDz8Saryz6LsD-foeC'}


In [6]:
client.create_run_from_pipeline_func(add_and_minus, 
            arguments={}, 
            experiment_name=EXPERIMENT_NAME
)

RunPipelineResult(run_id=9cd84c49-4ffc-40b7-841a-192cb67c8957)